In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
product_interest = 'tablet'
URL_BASE = 'https://www.olx.com.br/estado-sp/grande-campinas?q={product_interest}&o={page}'

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}
city_find = ['campinas', 'hortolândia', 'sumaré', 'valinhos', 'vinhedo', 'jundia', 'monte mor', 'indaiatuba', 'americana', 'nova odessa']

In [3]:
def check_end_find(page: BeautifulSoup) -> None:
    end = page.find('div', class_='sc-4c1746fa-0 hAnZTy')
    if end and 'ops!' in end.text.lower():
        raise Exception('End of search')
    
def get_products_per_page(page: int) -> BeautifulSoup:
    response = get(URL_BASE.format(page=page, product_interest=product_interest), headers=headers)
    html = response.text
    return BeautifulSoup(html, 'html.parser')

def process_page(page: BeautifulSoup) -> None:
    class_table = 'sc-e859a9d3-2 PnTvA'
    check_end_find(page)
    table = page.find('div', class_=class_table)
    if table and (rows := table.find_all('div', class_='sc-e859a9d3-0 gHnzLR renderIfVisible')):
        for row in rows:
            if row.find('img'):
                locate = row.find('div', class_='olx-ad-card__location-date-container')
                product = row.find('img')['alt']
                link = row.find('a')['href']
                price = row.find('div', class_='olx-ad-card__details-price--horizontal')
                if locate and locate.text.split(',')[0].lower() in city_find and product_interest in product.lower():
                    print(f"{locate.text[:50]:<50}|{price.text[:20]:>20}|{link}")


In [4]:
for page in range(1, 200):
    content_page = get_products_per_page(page)
    process_page(content_page)

Americana, Vila Cordenonsi                        |R$ 1.499R$ 1.3503x d|https://sp.olx.com.br/grande-campinas/informatica/tablets-e-readers/tablet-samsung-s6-lite-1267944086
Campinas, Vila Marieta                            |              R$ 300|https://sp.olx.com.br/grande-campinas/informatica/tablets-e-readers/tablet-multilaser-m10-3g-32gb-10-1-32gb-preto-e-2gb-de-memoria-ram-1269963771
Campinas, Jardim Márcia                           |              R$ 150|https://sp.olx.com.br/grande-campinas/informatica/tablets-e-readers/tablet-dl-r-150-1269930002
Campinas, Swiss Park                              |            R$ 2.999|https://sp.olx.com.br/grande-campinas/informatica/tablets-e-readers/tablet-panasonic-rugged-fz-g1-1269914396
Indaiatuba, Vila Pires da Cunha                   |R$ 5003x de R$ 166,6|https://sp.olx.com.br/grande-campinas/celulares/troco-por-tablet-1269890965
Campinas, Jardim Aurélia                          |              R$ 800|https://sp.olx.com.br/grande-campinas/in

Exception: End of search